# Package

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
from sklearn.utils import shuffle
import pickle

d:\Portofolio Data science\Time Series\Travaux pratiques Bases\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importation des données

In [3]:
df_stationary_train = pd.read_csv("df_stationary_train.csv", index_col="date")
df_stationary_test = pd.read_csv("df_stationary_test.csv", index_col="date")

In [4]:
df_stationary_train.head()

,UNRATE,TB3MS,RPI,INDPRO,DPCERA3M086SBEA,S&P 500,BUSLOANS,CPIAUCSL,OILPRICEx,M2SL,USREC
date,,,,,,,,,,,
1960-01-01,-0.8,0.30,0.020977,0.091980,0.001204,0.017909,0.011578,-0.006156,0.0,0.001323,0
1960-02-01,-1.1,-0.19,0.014565,0.076964,0.006009,-0.025663,0.011905,-0.003767,0.0,0.002007,0
1960-03-01,-0.2,-1.18,0.006250,0.007961,0.021240,-0.070857,-0.008356,-0.005455,0.0,0.001324,0
1960-04-01,0.0,-1.12,0.006489,-0.025915,0.033752,-0.040442,-0.009098,0.005090,0.0,0.000634,1
1960-05-01,0.0,-0.67,0.007747,-0.018121,0.009040,-0.010090,-0.000359,0.003383,0.0,0.003977,1


In [5]:
df_stationary_train.index = pd.to_datetime(df_stationary_train.index)

# stationary

# 1) Paramètres & sous-ensemble TRAIN global

In [22]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

# ---------- Paramètres ----------
h = 12                     # horizon de prévision (mois)
step_size = 12             # refit annuel
winsor_level = 0.01
df_full = df_stationary_train.copy()  # index mensuel trié
df_train_global = df_full.loc["1960-01":"1989-12"].copy()

# ✅ Inclure toutes les features (aucune exclusion)
cols_tx = [c for c in df_train_global.columns if c != "UNRATE"]

# ---------- Containers ----------
models = []
coefs = []
intercepts = []
train_periods = []
forecast_records = []  # (origin, target, y_true, y_hat, n_train)


In [29]:
# ---------- Boucle EXPANDING WINDOW ----------
n_total = len(df_train_global)
for t, end in enumerate(range(step_size, n_total + 1, step_size)):
    df_train_local = df_train_global.iloc[:end].copy()   # ✅ corrige .iloc et .copy()

    # Assez d'obs pour aligner X_t avec Y_{t+h} ?
    if len(df_train_local) <= h:
        continue

    # X_train : jusqu'à t_end - h ; Y_train : UNRATE décalé de -h sur la même plage
    X_train = df_train_local[cols_tx].iloc[:-h].copy()
    Y_train = df_train_local["UNRATE"].shift(-h).iloc[:-h].copy()

    # Retire lignes avec NaN éventuels
    valid = ~(X_train.isnull().any(axis=1) | Y_train.isnull())
    X_train = X_train.loc[valid]
    Y_train = Y_train.loc[valid]

    # 1️⃣ Winsorisation (1% - 99%) apprise sur la fenêtre courante
    lower_wins = X_train.quantile(winsor_level)
    upper_wins = X_train.quantile(1 - winsor_level)
    Xw = X_train.clip(lower=lower_wins, upper=upper_wins, axis=1)

    # 2️⃣ Normalisation (toujours activée)
    mean_train = Xw.mean()
    std_train = Xw.std().replace(0, 1)
    Xs = (Xw - mean_train) / std_train

    # 3️⃣ Entraînement OLS
    model = LinearRegression()
    model.fit(Xs, Y_train)

    # 4️⃣ Prévision pseudo-OOS à l'origine t_end : ŷ_{t_end+h}
    #    -> on prend la dernière observation dispo (t_end) côté X
    x_origin = df_train_local[cols_tx].iloc[[-1]].copy()
    x_origin_w = x_origin.clip(lower=lower_wins, upper=upper_wins, axis=1)
    x_origin_s = (x_origin_w - mean_train) / std_train
    y_hat = float(model.predict(x_origin_s)[0])

    origin_date = df_train_local.index[-1]
    # date cible = origin + h si dispo dans df_full
    idx_origin = df_full.index.get_loc(origin_date)
    target_date = df_full.index[idx_origin + h] if idx_origin + h < len(df_full) else pd.NaT
    y_true = float(df_full.loc[target_date, "UNRATE"]) if pd.notna(target_date) else np.nan

    # 5️⃣ Sauvegarde
    models.append(model)
    coefs.append(pd.Series(model.coef_, index=cols_tx))
    intercepts.append(model.intercept_)
    train_periods.append(origin_date)
    forecast_records.append({
        "origin": origin_date, "target": target_date,
        "y_true": y_true, "y_hat": y_hat, "n_train": len(Xs)
    })

    print(f"[{t:02d}] Fin {origin_date.strftime('%Y-%m')} | train={len(Xs)} | "
          f"ŷ({('NA' if pd.isna(target_date) else target_date.strftime('%Y-%m'))}) = {y_hat:.3f}")

[01] Fin 1961-12 | train=12 | ŷ(1962-12) = 2.466
[02] Fin 1962-12 | train=24 | ŷ(1963-12) = -0.489
[03] Fin 1963-12 | train=36 | ŷ(1964-12) = -0.571
[04] Fin 1964-12 | train=48 | ŷ(1965-12) = -0.928
[05] Fin 1965-12 | train=60 | ŷ(1966-12) = -0.478
[06] Fin 1966-12 | train=72 | ŷ(1967-12) = -0.412
[07] Fin 1967-12 | train=84 | ŷ(1968-12) = -0.704
[08] Fin 1968-12 | train=96 | ŷ(1969-12) = -0.869
[09] Fin 1969-12 | train=108 | ŷ(1970-12) = 0.650
[10] Fin 1970-12 | train=120 | ŷ(1971-12) = 0.780
[11] Fin 1971-12 | train=132 | ŷ(1972-12) = -0.210
[12] Fin 1972-12 | train=144 | ŷ(1973-12) = -0.908
[13] Fin 1973-12 | train=156 | ŷ(1974-12) = 1.567
[14] Fin 1974-12 | train=168 | ŷ(1975-12) = 2.097
[15] Fin 1975-12 | train=180 | ŷ(1976-12) = -0.305
[16] Fin 1976-12 | train=192 | ŷ(1977-12) = -0.194
[17] Fin 1977-12 | train=204 | ŷ(1978-12) = -0.156
[18] Fin 1978-12 | train=216 | ŷ(1979-12) = 0.318
[19] Fin 1979-12 | train=228 | ŷ(1980-12) = 0.462
[20] Fin 1980-12 | train=24

In [31]:
for t, end in enumerate(range(step_size, n_total + 1, step_size)):
    print(t)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [24]:
# ===================== ÉVALUATION PSEUDO–OUT-OF-SAMPLE =====================
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

# 1️⃣ Rassembler les prévisions
forecast_df = pd.DataFrame(forecast_records).dropna(subset=["y_true", "y_hat"]).copy()
forecast_df = forecast_df.sort_values("target").reset_index(drop=True)

def r2_origin_reg(y, yhat):
    """R² d'une régression à l’origine: y ≈ b * yhat (sans intercept)."""
    denom = np.dot(yhat, yhat)
    if denom == 0:
        return np.nan
    b = np.dot(yhat, y) / denom
    sse = np.sum((y - b * yhat) ** 2)
    sst = np.sum((y - y.mean()) ** 2)
    return 1 - sse / sst if sst > 0 else np.nan

def corr_pvalue(y, yhat):
    if len(y) < 3:
        return np.nan, np.nan
    r, p = pearsonr(y, yhat)
    return float(r), float(p)

if forecast_df.empty:
    print("\n[ÉVALUATION] Aucune prévision disponible (forecast_df est vide).")
else:
    y = forecast_df["y_true"].values
    yhat = forecast_df["y_hat"].values

    # 2️⃣ Métriques principales
    r2   = r2_score(y, yhat)
    mae  = mean_absolute_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))   # ✅ compatible toutes versions
    r2_o = r2_origin_reg(y, yhat)
    corr, pval = corr_pvalue(y, yhat)
    amd  = float(abs(np.mean(y - yhat)))          # biais absolu moyen

    # 3️⃣ Benchmark naïf (utile si la cible est Δ12 UNRATE)
    yhat_naive0 = np.zeros_like(y)
    mae_naive0  = mean_absolute_error(y, yhat_naive0)
    rmse_naive0 = np.sqrt(mean_squared_error(y, yhat_naive0))

    # 4️⃣ Précision directionnelle
    hit_rate = float(np.mean(np.sign(y) == np.sign(yhat))) if len(y) > 0 else np.nan

    # 5️⃣ Calibration (Mincer–Zarnowitz): y = a + b * yhat
    reg = LinearRegression()
    reg.fit(yhat.reshape(-1, 1), y)
    a_calib = float(reg.intercept_)
    b_calib = float(reg.coef_[0])

    # 6️⃣ Résumé par décennie (facultatif)
    by_decade = (
        forecast_df.assign(decade=forecast_df["target"].dt.year // 10 * 10)
                   .groupby("decade")
                   .apply(lambda g: pd.Series({
                       "n": len(g),
                       "MAE": mean_absolute_error(g["y_true"], g["y_hat"]),
                       "RMSE": np.sqrt(mean_squared_error(g["y_true"], g["y_hat"]))
                   }))
                   .reset_index()
    )

    # 7️⃣ Impression des résultats
    print("\n=== ÉVALUATION PSEUDO–OOS (h = {} mois) ===".format(h))
    print(f"R²            : {r2:.3f}")
    print(f"R² (origin)   : {r2_o:.3f}")
    print(f"MAE           : {mae:.3f}")
    print(f"RMSE          : {rmse:.3f}")
    print(f"Corr(y, ŷ)   : {corr:.3f}  (p={pval:.3g})")
    print(f"Hit rate sign : {hit_rate:.3f}")
    print(f"AMD (|bias|)  : {amd:.3f}")

    print("\n--- Benchmark naïf (zéro-changement) ---")
    print(f"MAE_naïf0     : {mae_naive0:.3f}")
    print(f"RMSE_naïf0    : {rmse_naive0:.3f}")

    if not by_decade.empty:
        print("\n--- MAE/RMSE par décennie ---")
        print(by_decade.to_string(index=False))

    # 8️⃣ Sauvegarde des résultats
    eval_results = {
        "overall": {
            "r2": r2, "r2_origin": r2_o, "mae": mae, "rmse": rmse,
            "corr": corr, "pval": pval, "hit_rate": hit_rate, "amd": amd
        },
        "benchmark_naive0": {
            "mae": mae_naive0, "rmse": rmse_naive0
        },
        "calibration": {
            "intercept": a_calib, "slope": b_calib
        },
        "by_decade": by_decade,
        "forecast_df": forecast_df
    }

    print("\n✅ Évaluation terminée. Résultats enregistrés dans eval_results.")


=== ÉVALUATION PSEUDO–OOS (h = 12 mois) ===
R²            : 0.095
R² (origin)   : 0.205
MAE           : 0.809
RMSE          : 1.058
Corr(y, ŷ)   : 0.453  (p=0.0154)
Hit rate sign : 0.714
AMD (|bias|)  : 0.010

--- Benchmark naïf (zéro-changement) ---
MAE_naïf0     : 0.800
RMSE_naïf0    : 1.113

--- MAE/RMSE par décennie ---
 decade    n      MAE     RMSE
   1960  8.0 0.695125 1.136080
   1970 10.0 0.772075 0.930614
   1980 10.0 0.937675 1.112530

✅ Évaluation terminée. Résultats enregistrés dans eval_results.


C:\Users\Mita\AppData\Local\Temp\ipykernel_14108\1481986021.py:60: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [25]:
# ==========================================================
# 🔍 IMPORTANCE PAR PERMUTATION — PSEUDO-OOS
# ==========================================================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

def permutation_importance_pseudo_oos(models, df_train_global, cols_tx, h=12, n_repeats=20, metric=mean_absolute_error):
    """
    Importance par permutation pour une série de modèles OLS entraînés
    selon une logique expanding window pseudo–OOS.
    -> aucune réestimation
    -> mesure la dégradation moyenne de la performance après permutation de chaque variable
    """
    var_imp = {col: [] for col in cols_tx}

    for i, model in enumerate(models):
        # Reconstituer la fenêtre utilisée par le modèle i
        end_idx = (i + 1) * 12  # correspond à ton step_size = 12
        df_win = df_train_global.iloc[:end_idx].copy()

        if len(df_win) <= h:
            continue

        # Préparation des données (alignement X_t avec Y_{t+h})
        X = df_win[cols_tx].iloc[:-h].copy()
        y = df_win["UNRATE"].shift(-h).iloc[:-h].copy()
        valid = ~(X.isnull().any(axis=1) | y.isnull())
        X, y = X.loc[valid], y.loc[valid]

        # Score de base (MAE sur données d'origine)
        base_score = metric(y, model.predict(X))

        # Boucle sur chaque variable
        for col in cols_tx:
            perm_scores = []
            for _ in range(n_repeats):
                X_perm = X.copy()
                X_perm[col] = np.random.permutation(X_perm[col])
                perm_scores.append(metric(y, model.predict(X_perm)))
            perm_scores = np.array(perm_scores)
            var_imp[col].append(np.mean(perm_scores) / base_score)

    # Agrégation moyenne sur toutes les fenêtres
    results = []
    for col, ratios in var_imp.items():
        if len(ratios) > 0:
            results.append({
                "variable": col,
                "perm_mae_ratio_mean": np.mean(ratios),
                "perm_mae_ratio_std": np.std(ratios),
                "n_windows": len(ratios)
            })

    imp_df = pd.DataFrame(results).sort_values("perm_mae_ratio_mean", ascending=False).reset_index(drop=True)
    return imp_df

In [26]:
# ----------------------------------------------------------
# 🧭 Appel de la fonction sur ton jeu de modèles OLS
# ----------------------------------------------------------
perm_df = permutation_importance_pseudo_oos(
    models=models,
    df_train_global=df_train_global,
    cols_tx=cols_tx,
    h=h,
    n_repeats=20  # augmente à 50 pour des résultats plus stables
)

print("\n=== 🔍 Importance par permutation (pseudo–OOS) ===")
print(perm_df.head(15).to_string(index=False))


=== 🔍 Importance par permutation (pseudo–OOS) ===
       variable  perm_mae_ratio_mean  perm_mae_ratio_std  n_windows
          USREC             1.089988            0.055342         28
          TB3MS             1.028684            0.025290         28
        S&P 500             1.013657            0.004299         28
      OILPRICEx             1.004552            0.004326         28
DPCERA3M086SBEA             1.000241            0.000357         28
           M2SL             1.000235            0.000371         28
            RPI             1.000066            0.000359         28
       BUSLOANS             1.000058            0.000205         28
         INDPRO             1.000039            0.002126         28
       CPIAUCSL             1.000038            0.000153         28


Ton modèle OLS prédit principalement le chômage via le cycle économique :
- USREC (récession) est la variable clé — sa permutation dégrade la performance de ~9 %.
- TB3MS (taux court) joue un rôle secondaire.
- Les autres variables ont un effet négligeable.

Conclusion : le pouvoir prédictif du modèle vient surtout des variables cycliques (récession, taux), les autres apportent peu d’information.

In [27]:
# ==========================================================
# 💡 IMPORTANCE SHAPLEY (pour modèle OLS)
# ==========================================================
import shap
import numpy as np
import pandas as pd

# 1️⃣ On utilise le dernier modèle entraîné
model_final = models[-1]

# 2️⃣ On reconstitue ses données finales (dernière fenêtre du train)
df_final = df_train_global.copy()
X_full = df_final[cols_tx].iloc[:-h].copy()
Y_full = df_final["UNRATE"].shift(-h).iloc[:-h].copy()
valid = ~(X_full.isnull().any(axis=1) | Y_full.isnull())
X_full = X_full.loc[valid]
Y_full = Y_full.loc[valid]

# 3️⃣ Calcul des valeurs SHAP
# Pour les modèles linéaires, on peut utiliser shap.LinearExplainer (plus stable)
explainer = shap.LinearExplainer(model_final, X_full, feature_perturbation="interventional")
shap_values = explainer(X_full)

# 4️⃣ Importance moyenne absolue
shap_df = pd.DataFrame({
    "variable": X_full.columns,
    "shap_mean_abs": np.abs(shap_values.values).mean(axis=0),
})
shap_df["shap_share"] = shap_df["shap_mean_abs"] / shap_df["shap_mean_abs"].sum()
shap_df = shap_df.sort_values("shap_mean_abs", ascending=False).reset_index(drop=True)

# 5️⃣ Affichage
print("\n=== 💡 Importance SHAP (shares) ===")
print(shap_df.head(10).to_string(index=False))


=== 💡 Importance SHAP (shares) ===
       variable  shap_mean_abs  shap_share
          TB3MS       0.148710    0.485751
          USREC       0.124740    0.407453
        S&P 500       0.017688    0.057777
      OILPRICEx       0.012534    0.040941
DPCERA3M086SBEA       0.001011    0.003303
         INDPRO       0.000871    0.002845
       CPIAUCSL       0.000309    0.001010
           M2SL       0.000204    0.000667
            RPI       0.000043    0.000139
       BUSLOANS       0.000035    0.000114


d:\Portofolio Data science\Time Series\Travaux pratiques Bases\.venv\Lib\site-packages\shap\explainers\_linear.py:99: FutureWarning: The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, maskers.Partition or maskers.Impute).
  warnings.warn(wmsg, FutureWarning)


- TB3MS (0.49)	🟢 Représente ~49 % de l’influence totale du modèle. Le taux d’intérêt à 3 mois est donc la variable la plus déterminante pour les prévisions du chômage : quand les taux montent, le modèle anticipe souvent une hausse future du chômage.

- USREC (0.41)	🔵 Représente ~41 % de l’influence totale. Le dummy de récession (NBER) pèse presque autant : le simple fait d’être en récession ou non explique une large part des variations prévues du chômage.

- S&P 500, OILPRICEx 🟠 Poids faibles (~6 % et 4 %) : les conditions boursières et le prix du pétrole ont un impact marginal dans la version linéaire du modèle.

# 📊 Métriques utilisées

## 1) Performance globale

**Coefficient de détermination (R²)**  
$$
R^2 = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}
$$  
➡️ Part de la variance expliquée par le modèle (0 = pas mieux que la moyenne, 1 = parfait).

**Erreur absolue moyenne (MAE)**  
$$
MAE = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|
$$  
➡️ Écart absolu moyen entre valeurs réelles et prédites, robuste aux outliers.

**Erreur quadratique moyenne (RMSE)**  
$$
RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2}
$$  
➡️ Similaire au MAE mais pénalise davantage les grosses erreurs.

**Corrélation de Pearson**  
$$
\rho(y, \hat{y}) = \frac{\text{Cov}(y, \hat{y})}{\sigma_y \cdot \sigma_{\hat{y}}}
$$  
➡️ Mesure le degré de lien linéaire entre les prédictions et les observations.

**Abs Mean Deviance (AMD)**  
$$
AMD = \frac{1}{n}\sum_{i=1}^n |\hat{y}_i - \bar{\hat{y}}|
$$  
➡️ Écart moyen des prédictions par rapport à leur moyenne ; sert de référence pour la permutation prédiction-basée.

---

## 2) Importance par permutation
La relation entre Y et X dépend du temps. Quand on perturbe la série X (en la mélangeant), on casse ce lien, et si l’erreur augmente, cela montre que X est une variable clé pour expliquer Y.

**Ratio MAE**  
$$
PI^{MAE}_j = \frac{MAE^{(perm)}_j}{MAE^{(base)}}
$$  
➡️ Si > 1, la variable est utile pour réduire l’erreur absolue.

**Ratio RMSE**  
$$
PI^{RMSE}_j = \frac{RMSE^{(perm)}_j}{RMSE^{(base)}}
$$  
➡️ Si > 1, la variable aide à limiter les grosses erreurs.

**Déviance de prédiction**  
$$
PI^{dev}_j = \frac{1}{n}\sum_{i=1}^n \big|\hat{y}_i - \hat{y}^{(perm)}_{i,j}\big|
$$  
➡️ Mesure combien les prédictions changent quand on brouille une variable.

---

## 3) Importance Shapley

**Décomposition des prédictions**  
$$
\hat{y}_i = \phi_0 + \sum_{j=1}^p \phi_{ij}
$$  
➡️ Chaque prédiction est expliquée par une contribution \(\phi_{ij}\) par variable.

**Importance absolue moyenne**  
$$
\text{Mean}(|\phi_j|) = \frac{1}{n}\sum_{i=1}^n |\phi_{ij}|
$$  
➡️ Contribution moyenne (absolue) d’une variable sur toutes les prédictions.

**Shapley share**  
$$
\Gamma_j = \frac{\text{Mean}(|\phi_j|)}{\sum_{k=1}^p \text{Mean}(|\phi_k|)}
$$  
➡️ Part relative de la variable dans l’explication totale (somme des parts = 1).

## Interprétation des résultats 

### Performance globale 
- R² = 0.2266 → modèle OLS explique ~23 % de la variance du chômage US.
- MAE = 0.6774 → en moyenne, l’erreur absolue est de 0.68 points (dans l’unité de la variable cible).
- RMSE = 0.8750 → un peu plus élevé que le MAE, ce qui indique la présence de grosses erreurs ponctuelles.
- Corrélation = 0.4760 (p ≈ 10⁻³⁵) → lien positif et significatif entre prédictions et observations, mais seulement modéré. Ce qui peut expliquer la présence d'une relation 
- Abs Mean Deviance = 0.3370 → sert ici de référence pour l’importance prédiction-basée : les prédictions s’écartent en moyenne de 0.34 de leur propre moyenne.

Lecture : le modèle OLS capte une partie utile du signal, mais laisse beaucoup de variance inexpliquée. La corrélation faible illustre éventuellement la présence des relations non-linéaire, et non captées par OLS.

### 🔹 2. Importance par permutation
- INDPRO (Industrial Production) : la plus influente. Sa permutation augmente MAE de +19 % et RMSE de +20 %, avec une forte déviance de prédiction (0.41).
- TB3MS (Taux d’intérêt à 3 mois) : impact non négligeable, ratios ~1.02 et déviance ~0.10.
- BUSLOANS (Prêts commerciaux) : rôle similaire (MAE ratio 1.017, déviance ~0.09).
- S&P 500 : contribution modérée, ratios légèrement > 1.
- RPI, M2SL : influence plus faible mais perceptible.
- CPIAUCSL, OILPRICEx, DPCERA3M086SBEA : quasi neutres (ratios ≈ 1, déviance très faible).

Lecture : INDPRO domine largement la performance, les autres apportent des compléments mais plus modestes.

### 🔹 3. Importance Shapley (shares)
- INDPRO : ~52 % de l’explication totale des prédictions → cohérence parfaite avec la permutation.
- TB3MS (12 %) + BUSLOANS (12 %) : deux autres piliers importants.
- S&P 500 (9,7 %) : contribue de façon notable.
- M2SL (5 %), RPI (3 %), CPIAUCSL (3,5 %) : apports plus secondaires.
- OILPRICEx et DPCERA3M086SBEA (<2 %) : quasi négligeables dans ce modèle.

Lecture : INDPRO est la variable macroéconomique centrale, suivie par des indicateurs financiers (taux courts, prêts bancaires, marché actions).

# Test

In [28]:
import pickle
import pandas as pd

# 🔖 Nom du modèle
model_name = "OLS_h12_expanding_window"

# 🔹 Rassembler tout dans un dictionnaire structuré
exp_results = {
    "model_name": model_name,
    "model_type": "LinearRegression (OLS)",
    "description": "Modèle OLS avec fenêtre expanding, horizon h=12 mois.",
    "models": models,
    "coefs": coefs,
    "intercepts": intercepts,
    "train_periods": train_periods,
    "forecast_records": forecast_records,
    "params": {
        "h": h,
        "step_size": step_size,
        "winsor_level": winsor_level,
        "norm_var": True,
        "features": cols_tx
    }
}

# 🔸 Nom du fichier de sortie (automatique)
file_name = f"{model_name}.pkl"

# 💾 Sauvegarde
with open(file_name, "wb") as f:
    pickle.dump(exp_results, f)

print(f"✅ Modèle '{model_name}' sauvegardé sous '{file_name}'")

✅ Modèle 'OLS_h12_expanding_window' sauvegardé sous 'OLS_h12_expanding_window.pkl'
